<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#导入数据" data-toc-modified-id="导入数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>导入数据</a></span></li><li><span><a href="#模型提出" data-toc-modified-id="模型提出-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>模型提出</a></span><ul class="toc-item"><li><span><a href="#逻辑回归模型的假设函数" data-toc-modified-id="逻辑回归模型的假设函数-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>逻辑回归模型的假设函数</a></span></li><li><span><a href="#代价函数" data-toc-modified-id="代价函数-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>代价函数</a></span></li><li><span><a href="#梯度下降" data-toc-modified-id="梯度下降-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>梯度下降</a></span></li><li><span><a href="#多分类器(o-v-r-svms)" data-toc-modified-id="多分类器(o-v-r-svms)-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>多分类器(o-v-r svms)</a></span></li></ul></li><li><span><a href="#训练参数" data-toc-modified-id="训练参数-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>训练参数</a></span><ul class="toc-item"><li><span><a href="#检查各个数据" data-toc-modified-id="检查各个数据-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>检查各个数据</a></span></li><li><span><a href="#训练分类器" data-toc-modified-id="训练分类器-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>训练分类器</a></span></li></ul></li><li><span><a href="#预测标签" data-toc-modified-id="预测标签-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>预测标签</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.io import loadmat            # 将MATLAB的本机格式加载到Python

# 导入数据

In [2]:
path = 'data/ex3data1.mat'
data = loadmat(path)
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
print(data['X'].shape, data['y'].shape)

x, y = data['X'], data['y']

(5000, 400) (5000, 1)


x：400维“特征”是原始20 x 20图像中每个像素的灰度强度

y：10个数字类

一共有5000个样本


# 模型提出

## 逻辑回归模型的假设函数

sigmoid：$$g\left( z \right)=\frac{1}{1+{e^{-z}}}$$

合并后：$${h_\theta }\left( x \right)=\frac{1}{1+{e^{-{\theta ^T}X}}}$$

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def hyper(x, theta):
    x = np.mat(x)
    theta = np.mat(theta)
    
    temp = x * theta.T
    return sigmoid(temp)

## 代价函数
$$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{y^{(i)}}\log \left( {h_\theta }\left( {x^{(i)}} \right) \right)-\left( 1-{y^{(i)}} \right)\log \left( 1-{h_\theta }\left( {x^{(i)}} \right) \right)]}+\frac{\lambda }{2m}\sum\limits_{j=1}^n{\theta _j^2}$$

In [5]:
def cost(theta, x, y, learningRate):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    
    first = (-y).T * np.log(hyper(x, theta))
    second = (1-y).T * np.log(1-hyper(x,theta))
    left = (1/len(x))*(first - second)
    reg = (learningRate / (2*len(x))) * (np.sum(np.power(theta[:, 1:theta.shape[1]], 2)))
    
    return left+reg

## 梯度下降
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{\theta _0}:={\theta _0}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{h_\theta }\left( {x^{(i)}} \right)-{y^{(i)}}]x_{_0}^{(i)}} \\ 
 & \text{     }{\theta _j}:={\theta _j}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{h_\theta }\left( {x^{(i)}} \right)-{y^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{\theta _j} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}

In [6]:
def gradient(theta, x, y, learningRate):
    theta = np.matrix(theta)
    x = np.matrix(x)
    y = np.matrix(y)
    
    error = hyper(theta, x) - y.T
    dtheta =  (error.reshape((1, 5000)) * x) / len(x)
#     print(dtheta[2, :])
    dtheta[1:] += (learningRate / len(x)) * theta[1:]
#     print(dtheta[2, :])
    return dtheta

## 多分类器(o-v-r svms)
逻辑回归只能一次在2个类之间进行分类，所以要使用多个分类器

**一对一全分类**：
- 具有k个不同标签就有k个分类器，每个分类器在 “是 i” 和 “不是 i” 之间决定
- 计算k个分类器的最终权重，并将权重返回为k *（n + 1）数组
- 独立训练k个分类器，每次都使用全部的训练集训练

In [7]:
def one_vs_all(x, y, num_labels, learning_rate):
    rows = x.shape[0]
    x = np.insert(x, 0, values=np.ones(rows), axis=1)     # x = (5000, 401)
    params = x.shape[1]
    all_theta = np.zeros((num_labels, params))            # w[1] = (10, 401)
    
    # 独立训练每个分类器
    for i in range(1, num_labels+1):
        theta = np.mat(np.zeros(params))
        y_i = np.array([1 if ylabel==i else 0 for ylabel in y])            # 修改y值，变为0、1
        y_i = y_i.reshape(rows, 1)                        # y = (5000, 1)
        
        fmin = minimize(fun=cost, x0=theta, args=(x, y_i, learning_rate), method='TNC', jac=gradient)                        # 最小化cost
        all_theta[i-1, :] = fmin.x
        
    return all_theta
    

# 训练参数

## 检查各个数据

In [10]:
# 检查维度
rows = data['X'].shape[0]
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

# 两者效果一样
theta = np.zeros(params + 1)
theta = np.random.randn(params+1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [11]:
# 检查标签数
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

## 训练分类器

In [12]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-1.78831929e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.07814846e-03,  2.85950101e-07,  0.00000000e+00],
       [-3.56264869e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.69143541e-03, -5.41439968e-04,  0.00000000e+00],
       [-4.97162876e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -3.36285069e-05,  3.24337895e-07,  0.00000000e+00],
       ...,
       [-8.91481040e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.12925858e-04,  9.03523844e-06,  0.00000000e+00],
       [-4.88454791e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -5.55402245e-04,  3.69050052e-05,  0.00000000e+00],
       [-4.07194160e+00,  0.00000000e+00,  0.00000000e+00, ...,
         5.16413667e-04,  1.48344669e-05,  0.00000000e+00]])

# 预测标签
计算每个类的类概率，并输出类标签为具有最高概率的类


In [13]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

In [14]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.42%
